In [2]:
import os
import re
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage, Document
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.document_loaders import YoutubeLoader, TextLoader, GoogleDriveLoader #import especific loaders for each type of document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import LLMChain, SequentialChain, TransformChain, RetrievalQA
from langchain.chains import LLMMathChain, SQLDatabaseChain, SQLDatabaseSequentialChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import SQLDatabase
from langchain.agents import Tool, load_tools, initialize_agent

openai_api_key = os.environ.get("OPENAI_API_KEY")
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
environment_api_key = os.environ.get("ENVIRONMENT_API_KEY")

/home/victor/.local/lib/python3.8/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

prompt_history = [
    SystemMessage(content="You are a unhelpful assistant that makes a joke at whatever the user says"),
    HumanMessage(content="What day comes after friday?"),
    AIMessage(content="Not friday"),
    HumanMessage(content="And two days after that?")
]

result = chat(prompt_history)
print(result.content)

prompt_history.append(AIMessage(content = result.content))
prompt_history.append(HumanMessage(content="Repeat the first question I asked you"))

result = chat(prompt_history)
print(result.content)

# Find how to insert new messages into the existing chat history, like the AIMessage given as a response

Why don't you just tell me, smarty pants?
What day comes after friday?


In [3]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

system_template = """
You are a helpful assistant that {function}.
"""

user_template = """
{statement}. What is wrong in this statement?
"""

system_prompt_template = PromptTemplate(input_variables=["function"], template=system_template)
system_prompt = system_prompt_template.format(function = "makes a joke at whatever the user says")

user_prompt_template = PromptTemplate(input_variables=["statement"], template=user_template)
user_prompt = user_prompt_template.format(statement = "2 + 2 is equal to 5")

prompt_history = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt)
]

result = chat(prompt_history)
print(result.content)

Well, what's wrong is that it's a pretty bad joke. Nice try though! 😂 The correct answer is that it's a mathematically inaccurate statement.


In [9]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

response_schemas = [
    ResponseSchema(name="Question", description="Question given by the user"),
    ResponseSchema(name="Answer", description="Answer given by you"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# ---------------------------------------------------------------------------

system_template = """
You will be given a poorly formatted string from a user.
Correct all the mistakes.

{format_instructions}
"""

user_template = """
{statement}
"""

system_prompt_template = PromptTemplate(input_variables=["format_instructions"], template=system_template)
system_prompt = system_prompt_template.format(format_instructions = format_instructions)

user_prompt_template = PromptTemplate(input_variables=["statement"], template=user_template)
user_prompt = user_prompt_template.format(statement = " Thats is gud to knou")

prompt_history = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt)
]

print("SYSTEM PROMPT:")
print(system_prompt)

print("RESULT:")
print("")
result = chat(prompt_history)

result = output_parser.parse(result.content)
print(result)

print("")
print(result["Question"])
print(result["Answer"])

SYSTEM PROMPT:

You will be given a poorly formatted string from a user.
Correct all the mistakes.

The output should be a markdown code snippet formatted in the following schema:

```json
{
	"Question": string  // Question given by the user
	"Answer": string  // Answer given by you
}
```

RESULT:

{'Question': 'Thats is gud to knou', 'Answer': "That's good to know"}

Thats is gud to knou
That's good to know


In [ ]:
Document(page_content="Example of a paragraph of a text document",
         metadata={
            'document_id': "00001",
            'document_source' : "Source of the document to have as reference",
            'retrieval_time' : "05-04-2023"
         })

# Metadata can be used to filter documents by retrieval time, document source, etc.

In [41]:
# For different documents use its corresponding loader (ex. docxs, txt, PDF, URL, Drive, Mail, CSV, DataFrame...)
loader = YoutubeLoader.from_youtube_channel("https://www.youtube.com/watch?v=jcbEiZQ9B7o")
data = loader.load()

print(f"Found {len(data)} comments")
print("")

for x in data:
    print(f"Comment length: {len(x.page_content)}")
    print(x.page_content)
    x.metadata["date"] = "06-04-2023" 
    print(x.metadata)
    long_text = x

Found 1 comments

Comment length: 23519
This video is sponsored by Wren. Is love in art? Then it requires knowledge, and 
effort. Or is love a pleasant sensation, which to   experience is a matter of chance... something one 
falls into if one is lucky? This little book is   based on the former premise, while undoubtedly the 
majority of people today believe in the latter. not that people think that love is not important. 
They are starved for it. They watch endless numbers   of films and happy and unhappy love stories. They 
listen to hundreds of trashy songs about love. Yet   hardly anyone thinks that there's anything that 
needs to be learned about love. Scrolling through   posts about romantic relationships nowadays, I kind 
of hate it. Not because I'm a cynic or I find PDA to be cringy. In fact, I love, love. I think it should 
be central to the way we live our lives. But there   are several beliefs about romantic relationships 
that i see commonly floating around on the   internet

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
texts = text_splitter.split_documents([long_text])

print(f"Found {len(texts)} comments")
print("")

i = 1
for text in texts:
    print(f"Comment length: {len(text.page_content)}")
    text.metadata["order"] = i
    print(text.page_content)
    print(text.metadata)
    print("---------------------------")
    i += 1


Found 51 comments

Comment length: 489
This video is sponsored by Wren. Is love in art? Then it requires knowledge, and 
effort. Or is love a pleasant sensation, which to   experience is a matter of chance... something one 
falls into if one is lucky? This little book is   based on the former premise, while undoubtedly the 
majority of people today believe in the latter. not that people think that love is not important. 
They are starved for it. They watch endless numbers   of films and happy and unhappy love stories. They
{'source': 'jcbEiZQ9B7o', 'date': '06-04-2023', 'order': 1}
---------------------------
Comment length: 414
listen to hundreds of trashy songs about love. Yet   hardly anyone thinks that there's anything that 
needs to be learned about love. Scrolling through   posts about romantic relationships nowadays, I kind 
of hate it. Not because I'm a cynic or I find PDA to be cringy. In fact, I love, love. I think it should 
be central to the way we live our lives. But there

In [4]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)
embedding_text = embedding.embed_query("Hello, let's go to the beach!")

print("Vector length: " + str(len(embedding_text)))
print(embedding_text)

Vector length: 1536
[-0.008095895871520042, -0.002594479825347662, 0.004955783020704985, -0.02397678606212139, -0.01946248486638069, 0.026488877832889557, -0.021079177036881447, -0.01000483613461256, -0.014873567037284374, -0.02186265029013157, 0.014612408354878426, 0.013667265884578228, 0.014836258254945278, -0.0154580632224679, 0.012212242931127548, -0.02539450116455555, 0.025891944766044617, -0.01843028888106346, 0.02495923824608326, -0.003152549499645829, -0.018256183713674545, -0.015644604340195656, 0.01897747814655304, -0.0052107227966189384, -0.008618212305009365, -0.018417853862047195, 0.010147851891815662, -0.012218461371958256, 0.003864515805616975, -0.02713555470108986, 0.03439823165535927, 0.009749896824359894, -0.0007263455190695822, -0.0056646401062607765, -0.006852286867797375, -0.025195524096488953, -0.018069641664624214, -0.010054580867290497, 0.015955505892634392, -0.019798260182142258, -0.007449219468981028, -0.015022799372673035, -0.002507427241653204, -0.0193878691

In [5]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)
embedding_list = embedding.embed_documents([text.page_content for text in texts])

print(f"Found {len(embedding_list)} embeddings")
print("")

for embedding_document in embedding_list:
    print(embedding_document)

Found 51 embeddings

[0.006819365080446005, -0.014306171797215939, -5.014238922740333e-05, -0.008791351690888405, -0.01036894042044878, 0.010874577797949314, -0.011697081848978996, -0.024998720735311508, -0.018904102966189384, 0.0019871555268764496, -0.009108217433094978, 0.02685946598649025, -0.027857257053256035, 0.0005679994937963784, 0.004661978222429752, 0.014643263071775436, 0.040531907230615616, -0.007314889691770077, 0.0017183249583467841, -0.024486340582370758, -0.02352900058031082, -0.007523886859416962, -0.009411600418388844, -0.03883296251296997, 0.003950714599341154, -0.009667789563536644, 0.03279228135943413, -0.04368708282709122, -0.005208066664636135, -0.015924211591482162, 0.011892595328390598, -0.003023712430149317, -0.0147511325776577, -0.04390282183885574, -0.02338068000972271, -0.022490758448839188, -0.008710449561476707, 0.0012329128803685308, 0.005659769754856825, -0.007834010757505894, 0.022463791072368622, 0.00828571431338787, 0.005875508300960064, -0.033897940

In [43]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)
pinecone.init(api_key="04d65f27-278f-40f0-9cfb-c907b84115a7", environment="us-east4-gcp")
index_name="langchain"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension = 1536, metric = "cosine")

docsearch = Pinecone.from_texts([text.page_content for text in texts], embedding=embedding, index_name=index_name)

index = pinecone.Index(index_name)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 102}},
 'total_vector_count': 102}


In [40]:
query = "TikTok and social media"
docs = docsearch.similarity_search(query, k=3, include_metadata=True)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)
    print("------------------------------")

percent right, you got me, which is why proper   loving is counter culture. It's near impossible 
because it's so opposed to how current culture   is. How can I focus on giving when I'm 
surrounded by people on social media   showing off what they receive? When our culture 
is so based on consumerism, on who has what, and   obviously I'm going to be obsessed with increasing 
my social value when that's how I'm constantly   judged... How can I dedicate that much time
{}
------------------------------
that these TikToks would say, even if he shows   you that he loves you every other day, if he misses 
the mark on this one commercialized, arbitrary day,   then sorry he clearly doesn't love you. There is 
so much concern around what can my man do for me,  can he meet my standards, that this one day can 
determine the trajectory of the rest of your   relationship? I know it's different for everyone,
{}
------------------------------
create character tropes all the time, that we then   form 

In [10]:
#QA retrieval chain

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
embedding = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

pinecone.init(api_key="04d65f27-278f-40f0-9cfb-c907b84115a7", environment="us-east4-gcp")
index = pinecone.Index("langchain")

vectordb = Pinecone(index=index, embedding_function=embedding.embed_query, text_key="text")

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectordb.as_retriever())
query = "Who is sponsoring the video?"
qa.run(query)

'Wren is sponsoring the video.'

In [21]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

template = """
You are a poet. Given the title of the poem and a thematic color you have to write a beautiful short poem.

Title: {title}
Color: {color}
You:
"""

prompt_template = PromptTemplate(input_variables=["title", "color"], template=template)
poem_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="poem")

#-------------------------------------------------------------------

template = """
You are a poet critic. Given a poem you have to write a review for that poem.

Title: {title}
Poem: {poem}
You:
"""

prompt_template = PromptTemplate(input_variables=["title", "poem"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

#-------------------------------------------------------------------

sequential_chain = SequentialChain(
    chains=[poem_chain, review_chain],
    input_variables= ["title", "color"],
    output_variables=["poem", "review"],
    verbose = True
)

result_chain = sequential_chain({"title":"The Sky", "color":"green"})

print("")
print("POEM:")
print(result_chain["poem"])
print("")
print("REVIEW:")
print(result_chain["review"])




> Entering new SequentialChain chain...

> Finished chain.

POEM:
The sky above, so vast and wide,
A canvas painted green with pride.
The trees below, their leaves so bright,
A symphony of color, a wondrous sight.

The green of nature, so pure and true,
A reminder of life, of all that's new.
The sky above, a never-ending dream,
A world of wonder, a timeless scheme.

So let us gaze upon this green expanse,
And marvel at the beauty of chance.
For in the sky, we find our peace,
A place of solace, a sweet release.

REVIEW:
"The Sky" is a beautiful poem that captures the essence of nature's beauty. The imagery of the vast and wide sky painted green with pride, and the trees below with their bright leaves, creates a symphony of colors that is truly wondrous. The poet's use of language is simple yet powerful, reminding us of the purity and truth of nature.

The poem's message is clear - to appreciate the beauty of nature and find solace in it. The sky is a never-ending dream, a world of won

In [27]:
#Summarize. Types of summarizing techniques:
    # map_reduce -> Every chuck of a splitted text is summarized
    # Refine -> Recursive summarization. Context is carried. 

llm = OpenAI(model_name="text-davinci-003", temperature=0.0, openai_api_key=openai_api_key) #turbo doesnt work here

loader = YoutubeLoader.from_youtube_channel("https://www.youtube.com/watch?v=jcbEiZQ9B7o")
long_text = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
texts = text_splitter.split_documents(long_text)

summarize_chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
result_chain = summarize_chain.run(texts)

print(result_chain)



> Entering new MapReduceDocumentsChain chain...
Prompt after formatting:
Write a concise summary of the following:


"This video is sponsored by Wren. Is love in art? Then it requires knowledge, and 
effort. Or is love a pleasant sensation, which to   experience is a matter of chance... something one 
falls into if one is lucky? This little book is   based on the former premise, while undoubtedly the 
majority of people today believe in the latter. not that people think that love is not important. 
They are starved for it. They watch endless numbers   of films and happy and unhappy love stories. They"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"listen to hundreds of trashy songs about love. Yet   hardly anyone thinks that there's anything that 
needs to be learned about love. Scrolling through   posts about romantic relationships nowadays, I kind 
of hate it. Not because I'm a cynic or I find PDA to be cringy. In fact, I love, love. I thin

Token indices sequence length is longer than the specified maximum sequence length for this model (2807 > 1024). Running this sequence through the model will result in indexing errors




> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


" This video sponsored by Wren explores the idea of love in art, and whether it is something that requires knowledge and effort, or something that is a matter of chance. The majority of people today believe it is a matter of chance, but they still crave love and watch films and stories about it.

 Despite the abundance of love songs and posts about romantic relationships, there is still much to learn about love. The author does not hate love, but believes there are several misconceptions about it that should be addressed.

 This article discusses the various types of relationship advice that is commonly seen on the internet, particularly from straight women, which can be unhelpful and lead to questioning healthy relationships.

 This article discusses two fundamental concepts of love: whether it is passive or active, and whether it requires effort to bring about. It applies to all ro

In [21]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("What is 13 raised to the .3432 power?")

pattern = r"[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?"
number = re.findall(pattern, result)
print("---------------------------------------")

if number:
    float_number = float(number[0])
    print(float_number)
else:
    print("No number found in the input string.")

/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```python
print(13 ** 0.3432)
```

Answer: 2.4116004626599237

> Finished chain.
---------------------------------------
2.4116004626599237


In [6]:
db = SQLDatabase.from_uri("sqlite:///Camera_Store.db")
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, return_intermediate_steps=True)
db_result = db_chain("find all the camera models with resolutin 22 or higher, and add their prices")

print("")
print(db_result["result"])
print("-------------------------------------")
print(db_result["intermediate_steps"][-1])



/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new SQLDatabaseChain chain...
find all the camera models with resolutin 22 or higher, and add their prices 
SQLQuery:SELECT Model, Price FROM Camera WHERE Resolution >= '22'
SQLResult: [('Sony Alpha a6000', '550'), ('Canon EOS 5D Mark III', '2000')]
Answer:The camera models with resolution 22 or higher are the Sony Alpha a6000 and the Canon EOS 5D Mark III, with prices of 550 and 2000, respectively.
> Finished chain.

The camera models with resolution 22 or higher are the Sony Alpha a6000 and the Canon EOS 5D Mark III, with prices of 550 and 2000, respectively.
-------------------------------------
[('Sony Alpha a6000', '550'), ('Canon EOS 5D Mark III', '2000')]


In [8]:
#Useful in cases where the number of tables in the database is large. Can't use return_intermediary_steps.

db = SQLDatabase.from_uri("sqlite:///Camera_Store.db")
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

db_chain = SQLDatabaseSequentialChain.from_llm(llm=llm, database=db, verbose=True)
db_chain.run("What is the price of the Sony camera?")


/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Camera', 'Camera_features']

> Entering new SQLDatabaseChain chain...
What is the price of the Sony camera? 
SQLQuery:SELECT Price FROM Camera WHERE Model = 'Sony Alpha a6000'
SQLResult: [('550',)]
Answer:The price of the Sony camera is $550.
> Finished chain.

> Finished chain.


'The price of the Sony camera is $550.'

In [22]:
def remove_extra_spaces(inputs: dict) -> dict:
    words = inputs["text"].split()
    cleaned_text = ' '.join(words)
    return {"output_text" : cleaned_text}

space_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=remove_extra_spaces)
space_chain.run("A random   text   with a lot of    espaces")

'A random text with a lot of espaces'

In [37]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

template = """
You are a helpful translator. Given a text, you output the language it's written in.

text: {output_text}
You:
"""

prompt_template = PromptTemplate(input_variables=["output_text"], template=template)
translate_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="translation")

#-------------------------------------------------------------------

sequential_chain = SequentialChain(
    chains=[space_chain, translate_chain],
    input_variables= ["text"],
    output_variables=["translation"],
    verbose = True
)

result_chain = sequential_chain({"text":"Buenos    dias    señora"})

print(result_chain["translation"])



> Entering new SequentialChain chain...

> Finished chain.
Spanish


In [35]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
serpapi_api_key = "d3a3046889befef05ffa0d05ed25907fa76a057ec372bc17bd3f895243127ca9"

toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key) # Use "wikipedia", "wolfram" or "human" tools instead as alternative
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("What was the first album of the band that Natalie Bergman is a part of?")

print(response)

/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new AgentExecutor chain...
I'm not sure which band Natalie Bergman is in, so I should search for that first.
Action: Search
Action Input: "Natalie Bergman band"
Observation: Natalie Bergman is an American singer-songwriter. She is one half of the duo Wild Belle, along with her brother Elliot Bergman. Her debut solo album, Mercy, was released on Third Man Records on May 7, 2021. She is based in Los Angeles.
Thought:Okay, so Natalie Bergman is in the band Wild Belle. Now I need to search for their first album.
Action: Search
Action Input: "Wild Belle first album"
Observation: Isles
Thought:So the first album of the band Wild Belle is Isles. 
Thought: I now know the final answer.
Final Answer: The first album of the band that Natalie Bergman is a part of is Isles by Wild Belle.

> Finished chain.
The first album of the band that Natalie Bergman is a part of is Isles by Wild Belle.


In [9]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

toolkit = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("4 elevated to 13")

print(response)



> Entering new AgentExecutor chain...
I need to calculate the result of 4 raised to the power of 13
Action: Calculator
Action Input: 4^13
Observation: Answer: 67108864

Thought:That's the final answer to the question. 
Final Answer: 67108864

> Finished chain.
67108864


In [15]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
serpapi_api_key = "d3a3046889befef05ffa0d05ed25907fa76a057ec372bc17bd3f895243127ca9"

toolkit = load_tools(["serpapi","python_repl"],  serpapi_api_key=serpapi_api_key, llm=llm)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("Create a python function that sums to numbers given as input")

print(response)



> Entering new AgentExecutor chain...
I need to write a function that takes two numbers as input and returns their sum. I can do this easily with Python.
Action: Python REPL
Action Input: 
```
def sum_numbers(num1, num2):
    return num1 + num2

print(sum_numbers(5, 7))
```
Observation: 12

Thought:The function works as expected and returns the sum of the two input numbers.
Final Answer: The Python function that sums two numbers is:

```
def sum_numbers(num1, num2):
    return num1 + num2
```

> Finished chain.
The Python function that sums two numbers is:

```
def sum_numbers(num1, num2):
    return num1 + num2
```


In [21]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
serpapi_api_key = "d3a3046889befef05ffa0d05ed25907fa76a057ec372bc17bd3f895243127ca9"

toolkit = load_tools(["serpapi","requests", "python_repl"],  serpapi_api_key=serpapi_api_key, llm=llm)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("Which is title of the latest tweet of Elon Musk?")

print(response)



> Entering new AgentExecutor chain...
I need to search for Elon Musk's Twitter account and find his latest tweet.
Action: Search
Action Input: "Elon Musk Twitter"
Observation: Elon Reeve Musk FRS is a business magnate and investor. He is the founder, CEO and chief engineer of SpaceX; angel investor, CEO and product architect of Tesla, Inc.; owner and CEO of Twitter, Inc.; founder of the Boring Company; co-founder of Neuralink and OpenAI; and president of the philanthropic Musk Foundation.
Thought:That's not what I was looking for. I need to refine my search query to find Elon Musk's Twitter account.
Action: Search
Action Input: "Elon Musk Twitter account"
Observation: Follow the Elon Musk Topic on Twitter. You'll see top Tweets from a number of accounts that are experts, fans or just tend to talk about Elon Musk on Twitter.
Thought:That's still not what I was looking for. I need to search for Elon Musk's actual Twitter account.
Action: Search
Action Input: "Elon Musk Twitter handle"


In [6]:
llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)
serpapi_api_key = "d3a3046889befef05ffa0d05ed25907fa76a057ec372bc17bd3f895243127ca9"

toolkit = load_tools(["terminal"],  serpapi_api_key=serpapi_api_key, llm=llm)
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)
response = agent.run("Which are the folders inside the ai4hri folder?")

print(response)



> Entering new AgentExecutor chain...
I should use the "ls" command to list the contents of the ai4hri folder.
Action: Terminal
Action Input: ls ai4hri/
Observation: Command 'ls ai4hri/' returned non-zero exit status 2.
Thought:I need to navigate to the ai4hri folder first before using the ls command.
Action: Terminal
Action Input: cd ai4hri/
Observation: Command 'cd ai4hri/' returned non-zero exit status 2.
Thought:I need to check my current directory first to see if I am already in the correct directory.
Action: Terminal
Action Input: pwd
Observation: /home/victor/catkin_ws/src/ai4hri/scripts

Thought:I need to navigate to the ai4hri folder first before using the ls command.
Action: Terminal
Action Input: cd ../ai4hri/
Observation: Command 'cd ../ai4hri/' returned non-zero exit status 2.
Thought:I should try using the absolute path to the ai4hri folder.
Action: Terminal
Action Input: ls /home/victor/catkin_ws/src/ai4hri/ai4hri/
Observation: Command 'ls /home/victor/catkin_ws/src/ai

In [40]:
#Creating tools for conversational agents

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

tool_description1 = """
Use this tool to answer user question about love. 
If the user states "ask Olivia" use this tool to get the answer. This tool can also be used for follow up questions from the user.
"""

tool_description2 = """
Use this tool to answer user question about cameras.
If the user states "ask the shopkeeper" use this tool to get the answer. This tool can also be used for follow up questions from the user.
"""

tool_description3 = """
Use this tool to detect in which language is the text written.
"""

tools = [Tool(func=qa.run, description=tool_description1, name="Olivia DB"), 
         Tool(func=db_chain.run, description=tool_description2, name="Cameras DB"), 
         Tool(func=translate_chain.run, description=tool_description3, name="Language tool")
        ]

conversational_agent = initialize_agent(
    agent='zero-shot-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
)

result = conversational_agent("ask olivia what is love")

print(result["output"])

/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/victor/.local/lib/python3.8/site-packages/langchain/llms/openai.py:608: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new AgentExecutor chain...
I should use the Olivia DB tool to answer this question.
Action: Olivia DB
Action Input: "What is love?"
Observation: The video and context provide different perspectives on what love is, with one premise being that love in art requires knowledge and effort, while the majority of people believe it is a matter of chance and a pleasant sensation. The context also discusses how people are starved for love and often search for a partner as a means to satisfy their desire to not be lonely, with this search being compared to a strategical exchange where people become commodities on a personality market. However, the context does not provide a clear definition of what love is.
Thought:I should ask Olivia for a more specific question about love to provide a better answer.
Action: Olivia DB
Action Input: "Can you provide a more specific question about love?"
Observation: What is the perspective shift that Fromm and Bell Hooks suggest in regards to our con

In [15]:
# See how you can keep asking the agent

result = conversational_agent("What is the price of the Sony camera? Ask the shopkeeper")
print(result["output"])



> Entering new AgentExecutor chain...
I need to use the Cameras DB tool to get the answer to this question.
Action: Cameras DB
Action Input: "What is the price of the Sony camera?"

> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Camera', 'Camera_features']

> Entering new SQLDatabaseChain chain...
What is the price of the Sony camera? 
SQLQuery:SELECT Price FROM Camera WHERE Model = 'Sony Alpha a6000'
SQLResult: [('550',)]
Answer:The price of the Sony camera is $550.
> Finished chain.

> Finished chain.

Observation: The price of the Sony camera is $550.
Thought:I have the answer to the user's question.
Final Answer: The price of the Sony camera is $550.

> Finished chain.
The price of the Sony camera is $550.


In [41]:
# See how you can keep asking the agent

result = conversational_agent("Buenos dias, como está usted hoy")
print(result["output"])



> Entering new AgentExecutor chain...
This is a greeting in Spanish, I should use the Language tool to detect the language.
Action: Language tool
Action Input: "Buenos dias, como está usted hoy"
Observation: Spanish
Thought:

ValueError: Could not parse LLM output: `I should respond in Spanish or ask if the user prefers English.
Action: None needed`